In [ ]:
import os
print(os.getcwd())

In [2]:
import os
import pickle as pkl

import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
# Replace this with the path to your dataset
#DATA_PATH = "https://raw.githubusercontent.com/owkin/PyDESeq2/main/datasets/synthetic/"
df=pd.read_csv('RNA_count_withmeta.csv',index_col=0)
df.head()

In [ ]:
metadata=df[['Group']]

In [ ]:
metadata

In [ ]:
counts_df=df.copy()
counts_df

In [8]:
counts_df2 = counts_df.loc[counts_df['Group'].isin([ 'Vehicle','Treatment']),:]

In [ ]:
counts_df2 

In [ ]:
metadata2 = metadata.loc[metadata['Group'].isin([ 'Vehicle','Treatment']),:]
metadata2

###분석

In [12]:
samples_to_keep = ~metadata.Group .isna()
counts_df2 = counts_df2.loc[samples_to_keep]
metadata2 = metadata2.loc[samples_to_keep]

In [13]:
genes_to_keep = counts_df2.columns[counts_df2.sum(axis=0) >= 10]
counts_df2 = counts_df2[genes_to_keep]

In [16]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df2,
    ref_level =['Group',  'Vehicle'],
    metadata=metadata2,
    design_factors="Group",
    refit_cooks=True,
    inference=inference,
)

In [17]:
dds.deseq2()

Fitting size factors...
... done in 0.03 seconds.

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<fro

In [ ]:
print(dds)

In [19]:
with open(os.path.join('./', "dds.pkl"), "wb") as f:
    pkl.dump(dds, f)

In [ ]:
##open pickle
with open(os.path.join('./', "dds.pkl"), "rb") as f:
    dds2 = pkl.load(f)

print(dds2)

In [ ]:
print(dds.varm["dispersions"])

In [ ]:
print(dds.varm["LFC"])

In [23]:
stat_res = DeseqStats(dds, inference=inference)

In [ ]:
stat_res.summary()

In [25]:
stat_res.results_df.to_csv("./DEG_results.csv")